# Basic Strategy for Dealing with Small Datasets¶

* Create a naive convnet model without regularization to set a baseline for what can be achieved.
* Data Augmentation: to prevent overfitting.
* Feature extraction with a pretrained network.
* Fine tuning a pretrained network.

In this notebook, we augment our training samples to prevent overfitting. This is done by transforming the training images to yield a larger training set. In Keras, this is done using the `ImageDataGenerator`class

```python
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0,
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)
```